In [1]:

import numpy as np
import pandas as pd
import os

In [2]:
model_checkpoint = "distilbert-base-uncased"
use_gpu = True
num_workers = 2
cpus_per_worker = 1

In [3]:
import ray

ray.init(
    runtime_env={
        "pip": [
            "datasets",
            "evaluate",
            "accelerate>=0.16.0",
            "transformers>=4.26.0",
            "torch>=1.12.0",
            "deepspeed",
        ]
    }
)


2023-05-07 21:53:37,898	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.16
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8266


In [4]:
from datasets import load_dataset
imdb_data = load_dataset("imdb")

Found cached dataset imdb (/home/aakankshasanctis/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
import ray.data

ray_datasets = ray.data.from_huggingface(imdb_data)
ray_datasets

{'train': Dataset(num_blocks=1, num_rows=25000, schema={text: string, label: int64}),
 'test': Dataset(num_blocks=1, num_rows=25000, schema={text: string, label: int64}),
 'unsupervised': Dataset(num_blocks=1, num_rows=50000, schema={text: string, label: int64})}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)


In [37]:
import pandas as pd
from ray.data.preprocessors import BatchMapper

def preprocess_batch(examples: pd.DataFrame):
    texts = examples['text'].tolist()
    # print("Type of input: " , type(texts))

    # Tokenize the input text
    ret = tokenizer(texts, padding=True, truncation=True, return_tensors="np")
    
    # Convert the labels to a numpy array
    
    # labels = np.array(labels)
    ret['input_ids'] = ret['input_ids'].tolist()
    ret['attention_mask'] = ret['attention_mask'].tolist()

    ret = {**examples, **ret}
    return pd.DataFrame.from_dict(ret)


# def preprocess_batch(batch):
#     results = {}
#     results['text'] = batch['text']
#     return pd.DataFrame.from_dict(results)

# def tokenized_inputs(example):
#     return tokenizer(example["text"], padding=True, truncation=True, return_tensors="np")

batch_encoder = BatchMapper(preprocess_batch, batch_format="pandas")




In [8]:
ray_datasets

{'train': Dataset(num_blocks=1, num_rows=25000, schema={text: string, label: int64}),
 'test': Dataset(num_blocks=1, num_rows=25000, schema={text: string, label: int64}),
 'unsupervised': Dataset(num_blocks=1, num_rows=50000, schema={text: string, label: int64})}

In [9]:
transformed_data = batch_encoder.transform(ray_datasets['test'])  

In [10]:
# !pip install ipywidgets

In [11]:
transformed_data


2023-05-07 21:53:48,219	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[BatchMapper]
2023-05-07 21:53:48,220	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(BatchMapper pid=23456) Type of input:  <class 'list'>
(BatchMapper pid=23456) Type of input:  <class 'list'>
(BatchMapper pid=23456) Type of input:  <class 'list'>
(BatchMapper pid=23456) Type of input:  <class 'list'>
(BatchMapper pid=23456) Type of input:  <class 'list'>
(BatchMapper pid=23456) Type of input:  <class 'list'>
(BatchMapper pid=23456) Type of input:  <class 'list'>


In [12]:
# tokenized_datasets = imdb_data.map(tokenized_inputs, batched=True)

In [13]:
# tokenized_datasets

In [14]:
# ray_datasets = ray.data.from_huggingface(tokenized_datasets)

In [15]:
# imdb_data

In [16]:
# tokens = tokenizer(imdb_data['train'][:10]['text'], padding=True, truncation=True, return_tensors="np")

In [17]:
# batch = imdb_data['train'][:10]
# # print(batch.keys())
# texts = batch['text']
# # print(len(texts))
# texts = batch['text']
# labels = batch['label']

# # # Tokenize the input text
# tokenized_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="np")

# # Convert the labels to a numpy array
# labels = np.array(labels)

# # Return a dictionary with the inputs and labels
# # results =  {'input_ids': (tokenized_inputs['input_ids']).tolist(),
# #         'attention_mask': (tokenized_inputs['attention_mask']).tolist(),
# #         'labels': labels}
# # print(len(results['input_ids']))
# # print(len(results['attention_mask']))
# # print(len(results['labels']))
# # print(results)
# # pd.DataFrame.from_dict(results)


In [19]:
# tokens.keys()

In [20]:
# ray_datasets

In [56]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, default_data_collator
import numpy as np
import torch
batch_size = 16
# num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
# metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
# model_name = model_checkpoint.split("/")[-1]
# validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
# name = f"{model_name}-finetuned-{task}"

from datasets import load_metric

    
def trainer_init_per_worker(train_dataset, eval_dataset = None, **config):
    print(f"Is CUDA available: {torch.cuda.is_available()}")
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

    torch.backends.cuda.matmul.allow_tf32 = True
    deepspeed = {
        "fp16": {
            "enabled": "auto",
            "initial_scale_power": 8,
        },
        "bf16": {"enabled": "auto"},
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": "auto",
                "betas": "auto",
                "eps": "auto",
            },
        },
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True,
            },
            "offload_param": {
                "device": "cpu",
                "pin_memory": True,
            },
            "overlap_comm": True,
            "contiguous_gradients": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "gather_16bit_weights_on_model_save": True,
            "round_robin_gradients": True,
        },
        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 10,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "wall_clock_breakdown": False,
    }
    args = TrainingArguments(
        "ray-bert-finetune-imdb",
        save_strategy="no",
        logging_steps=1,
        learning_rate=config.get("learning_rate", 2e-5),
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        label_names=["input_ids", "attention_mask"],
        num_train_epochs=config.get("epochs", 2),
        weight_decay=config.get("weight_decay", 0.01),
        fp16=True,
        # gradient_checkpointing=True,
        deepspeed=deepspeed,
        push_to_hub=False,
        # disable_tqdm=True,  # declutter the output a little
        # no_cuda=not use_gpu,  # you need to explicitly set no_cuda if you want CPUs
    )


    def compute_metrics(eval_pred):
        load_accuracy = load_metric("accuracy")
        load_f1 = load_metric("f1")
        
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
        f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
        return {"accuracy": accuracy, "f1": f1}


    trainer = Trainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        data_collator = default_data_collator,
    )

    print("Starting training")
    return trainer

In [57]:
# ray_datasets

In [58]:
from ray.train.huggingface import HuggingFaceTrainer
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.air.integrations.mlflow import MLflowLoggerCallback

trainer = HuggingFaceTrainer(
    trainer_init_per_worker=trainer_init_per_worker,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu, resources_per_worker={"GPU": 1, "CPU": 1}),
    datasets={
        "train": ray_datasets["train"],
        "evaluation": ray_datasets["test"],
    },
    run_config=RunConfig(
        callbacks=[MLflowLoggerCallback(experiment_name="ray-bert-finetune-imdb")],
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="loss",
            checkpoint_score_order="min",
        ),
    ),
    preprocessor=batch_encoder,
)

In [59]:
# from transformers import DataCollatorWithPadding
# data_collator = DataCollatorWithPadding(tokenizer)

In [60]:
import os
os.environ['RAY_memory_usage_threshold'] = "0.98"

In [61]:
result = trainer.fit()

(RayTrainWorker pid=25151) 2023-05-07 23:12:55,145	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=23711) Loading extension module utils... [repeated 3x across cluster]
(RayTrainWorker pid=23711) 2023-05-07 23:12:55,145	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=23711) 2023-05-07 23:12:55,145	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]


(pid=25087) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=25087) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(RayTrainWorker pid=23711) Time to load utils op: 0.00036787986755371094 seconds [repeated 3x across cluster]


(HuggingFaceTrainer pid=25087) 2023-05-07 23:12:55,508	INFO streaming_executor.py:83 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[BatchMapper] -> AllToAllOperator[RandomizeBlockOrder]
(HuggingFaceTrainer pid=25087) 2023-05-07 23:12:55,509	INFO streaming_executor.py:84 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)


...

100%|█████████▉| 1563/1564 [12:36<00:00,  2.07it/s]


(RayTrainWorker pid=25151) {'loss': 0.1363, 'learning_rate': 0.0, 'epoch': 2.0}


100%|██████████| 1564/1564 [12:37<00:00,  2.11it/s]
(RayTrainWorker pid=25151) /opt/conda/envs/hpml_1/lib/python3.9/site-packages/torch/nn/modules/module.py:1802: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
(RayTrainWorker pid=25151)   warnings.warn(


(RayTrainWorker pid=25151) {'train_runtime': 759.2204, 'train_samples_per_second': 32.929, 'train_steps_per_second': 2.06, 'train_loss': 0.21420998097685598, 'epoch': 2.0}


100%|██████████| 1564/1564 [12:39<00:00,  2.06it/s]
2023-05-07 23:26:33,360	ERROR checkpoint_manager.py:361 -- Result dict has no key: eval_loss. checkpoint_score_attr must be set to a key in the result dict. Valid keys are: ['loss', 'learning_rate', 'epoch', 'step', 'train_runtime', 'train_samples_per_second', 'train_steps_per_second', 'train_loss', 'timestamp', 'time_this_iter_s', 'should_checkpoint', 'done', 'training_iteration', 'trial_id', 'date', 'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore', 'iterations_since_restore', 'experiment_tag']
2023-05-07 23:26:36,010	INFO tune.py:945 -- Total run time: 829.05 seconds (829.03 seconds for the tuning loop).


In [ ]:
result

In [ ]:
# from ray.train.huggingface import HuggingFacePredictor
# from ray.train.batch_predictor import BatchPredictor
# import pandas as pd

# predictor = BatchPredictor.from_checkpoint(
#     checkpoint=best_result.checkpoint,
#     predictor_cls=HuggingFacePredictor,
#     task="text-classification",
#     device=0 if use_gpu else -1,  # -1 is CPU, otherwise device index
# )
# prediction = predictor.predict(ray_datasets["test"].map_batches(lambda x: x[["sentence"]]), num_gpus_per_worker=int(use_gpu))
# prediction.show()